Nama : Fernando
Email : fernandosianturi58@gmail.com
Address : Jl.Pahlawan 12 Kel.Bukit Merapin, Kec.Gerunggang, Kota Pangkalpinang, Bangka Belitung.

In [ ]:
pip install split-folders

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
!unzip /tmp/rockpaperscissors.zip -d /tmp

In [ ]:
import os
import splitfolders
base_directory = '/tmp/rockpaperscissors/rps-cv-images'
os.remove('/tmp/rockpaperscissors/rps-cv-images/README_rpc-cv-images.txt')
splitfolders.ratio(base_directory, output = base_directory, seed = 42, ratio=(.6, .4))

train_directory = os.path.join(base_directory,'train')
val_directory = os.path.join(base_directory, 'val')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

train_data_generator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_data_generator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

train_generator = train_data_generator.flow_from_directory(
        train_directory,  
        target_size=(150, 150),  
        batch_size=16,
        class_mode='categorical'
        )
 
validation_generator = test_data_generator.flow_from_directory(
        val_directory,
        target_size=(150, 150), 
        batch_size=16, 
        class_mode='categorical'
        )

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

class CustomCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get('accuracy') >= 0.97 :
      print('Accuray model over 97%, stopping training model !')
      self.model.stop_training = True

history = model.fit(
          train_generator,
          steps_per_epoch=30,
          epochs=20,
          validation_data=validation_generator, 
          validation_steps=5,  
          verbose=2,
          callbacks = [CustomCallBack()]
          )

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

if classes[0,0] != 0:
  print('Picture = Paper')
elif classes[0,1] != 0 :
  print('Picture = Rock')
else:
  print('Picture = Scissors')